In [707]:
#%pip install -U spaCy
#%pip install keras
#%python -m spacy download en_core_web_sm
#%pip install -U jax jaxlib
#%pip install clean-text
#%pip install --upgrade ipykernel
#da eseguire al primo utilizzo

In [708]:
import string
import spacy
import json
import re
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from spacy.lang.it.stop_words import STOP_WORDS
from spacy.lang.en import English
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
from sklearn.model_selection import GridSearchCV
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from pandas import read_csv
from sklearn.svm import SVC
from sklearn import metrics
from sklearn import svm
from sklearn import neural_network
from sklearn import ensemble
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from pandas import read_csv
import nltk
import random

#init spaCy
punctuations = string.punctuation
nlp = spacy.load("en_core_web_lg")
stop_words = spacy.lang.en.stop_words.STOP_WORDS
parser = English()

# Custom transformer using spaCy
class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        # Cleaning Text
        return [clean_text(text) for text in X]

    def fit(self, X, y=None, **fit_params):
        return self

    def get_params(self, deep=True):
        return {}

# Basic function to clean the text
def clean_text(text):
    # Removing spaces and converting text into lowercase
    translator = str.maketrans("", "", string.punctuation)
    text_without_punctuation = text.translate(translator)
    return text_without_punctuation.lower()
    #return text.strip().lower()

# Tokenizer function
def spacy_tokenizer(sentence):
    mytokens = parser(sentence)
    mytokens = [ word.text for word in mytokens ]
    # remove stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]
    # return preprocessed list of tokens
    return mytokens



In [709]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\perna\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [710]:
df = read_csv('../Dataset/final_dataset_sexist.csv')

X = df['text']
y = df['label_sexist']
print(y.value_counts())
df.set_index('ID')


not sexist    15146
sexist         4854
Name: label_sexist, dtype: int64


,text,label_sexist
ID,,
0,"In Nigeria, if you rape a woman, the men rape ...",not sexist
1,"Then, she's a keeper. 😉",not sexist
2,This is like the Metallica video where the poo...,not sexist
3,woman?,not sexist
4,I bet she wished she had a gun,not sexist
...,...,...
19995,girls really get fucked almost every weekend ?,not sexist
19996,The hatred for moslems are Reasonable and Just...,not sexist
19997,Now this is a woman who gets it. 👆,not sexist


In [711]:
vector = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,1))

In [712]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

kf = KFold(n_splits=10, shuffle=True, random_state=42)

for train_index, test_index in kf.split(X):
    # Divisione dei dati in fold di addestramento e di test
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
#y_test = y_test.reset_index(drop=True)

In [713]:
print(f"x_train shape: {X_train.shape}")  # (50, 4)
print(f"y_train shape: {y_train.shape}")  # (50, 4)  ==> 2d, so something is wrong
print(f"x_test shape: {X_test.shape}")  # (50,) => also bad
print(f"y_test shape: {y_test.shape}")


x_train shape: (18000,)
y_train shape: (18000,)
x_test shape: (2000,)
y_test shape: (2000,)


In [714]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.utils import class_weight


#Linear Support Vector Machine classifier

classifier = svm.LinearSVC(max_iter = 10000, class_weight= {"not sexist": 0.2, "sexist": 1})
# Multi-layer Perceptron classifier
#classifier = neural_network.MLPClassifier(hidden_layer_sizes=(512,),verbose=True,max_iter=10)
# Random Foreset
#classifier = ensemble.RandomForestClassifier(class_weight='balanced')

# Create the pipeline

pipe = Pipeline([("cleaner", predictors()),
('vectorizer', vector),
('classifier', classifier)])

# Addestramento del modello
pipe.fit(X_train, y_train)

Pipeline(steps=[('cleaner', <__main__.predictors object at 0x000001FB9B8549D0>),
                ('vectorizer',
                 CountVectorizer(tokenizer=<function spacy_tokenizer at 0x000001FB207FB940>)),
                ('classifier',
                 LinearSVC(class_weight={'not sexist': 0.2, 'sexist': 1},
                           max_iter=10000))])

In [715]:
#!pip install scikit-learn -U
import sklearn.metrics

In [716]:
predicted = pipe.predict(X_test)
sklearn.metrics.f1_score(y_test, predicted,average='weighted')

predicted = pipe.predict(X_test)

acc = metrics.accuracy_score(y_test, predicted)
print("Accuracy:", acc)
print("======================================================")
precision_macro = metrics.precision_score(y_test, predicted, average='macro')
recall_macro = metrics.recall_score(y_test, predicted, average='macro')
fm_macro = metrics.f1_score(y_test, predicted, average='macro')
print("P-macro={0}, R-macro={1}, F1-macro={2}".format(precision_macro, recall_macro, fm_macro))
print("======================================================")
precision_micro = metrics.precision_score(y_test, predicted, average='micro')
recall_micro = metrics.recall_score(y_test, predicted, average='micro')
fm_micro = metrics.f1_score(y_test, predicted, average='micro')
print("P-micro={0}, R-micro={1}, F1-micro={2}".format(precision_micro, recall_micro, fm_micro))
print("======================================================")
print(metrics.classification_report(y_test, predicted, digits=4))

Accuracy: 0.7875
P-macro=0.7154781493016786, R-macro=0.7415809838324693, F1-macro=0.7257532831365048
P-micro=0.7875, R-micro=0.7875, F1-micro=0.7875
              precision    recall  f1-score   support

  not sexist     0.8838    0.8297    0.8559      1521
      sexist     0.5472    0.6534    0.5956       479

    accuracy                         0.7875      2000
   macro avg     0.7155    0.7416    0.7258      2000
weighted avg     0.8031    0.7875    0.7936      2000



In [717]:
df1 = read_csv('../Dataset/final_dataset_category.csv')
X1 = df1['text']
y1 = df1['label_category']
df1.set_index('ID')
print(y1.value_counts())


2. derogation                               2271
3. animosity                                1665
4. prejudiced discussions                    475
1. threats, plans to harm and incitement     443
Name: label_category, dtype: int64


In [718]:
from sklearn.model_selection import train_test_split
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.20, random_state=42)
y1_test = y1_test.reset_index(drop=True)

In [719]:

from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')

vector1 = CountVectorizer(tokenizer = tokenizer.tokenize, ngram_range=(1,2))


In [728]:
from sklearn.naive_bayes import MultinomialNB

#Linear Support Vector Machine classifier
classifier = svm.LinearSVC(max_iter= 10000)
# Multi-layer Perceptron classifier
#classifier = neural_network.MLPClassifier(hidden_layer_sizes=(512,),verbose=True, max_iter=100)
# Random Foreset
#classifier = ensemble.RandomForestClassifier()

# Create the pipeline
pipe1 = Pipeline([("cleaner", predictors()),
('vectorizer', vector1),
('classifier', classifier)])

# Addestramento del modello
pipe1.fit(X1_train, y1_train)

Pipeline(steps=[('cleaner', <__main__.predictors object at 0x000001FB7156F1C0>),
                ('vectorizer',
                 CountVectorizer(ngram_range=(1, 2),
                                 tokenizer=<bound method PreTrainedTokenizer.tokenize of BertTokenizer(name_or_path='bert-large-uncased', vocab_size=30522, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)>)),
                ('classifier', LinearSVC(max_iter=10000))])

In [729]:
predicted = pipe1.predict(X1_test)
sklearn.metrics.f1_score(y1_test, predicted,average='weighted',zero_division=0)

predicted = pipe1.predict(X1_test)

acc = metrics.accuracy_score(y1_test, predicted)
print("Accuracy:", acc)
print("======================================================")
precision_macro = metrics.precision_score(y1_test, predicted, average='macro')
recall_macro = metrics.recall_score(y1_test, predicted, average='macro')
fm_macro = metrics.f1_score(y1_test, predicted, average='macro')
print("P-macro={0}, R-macro={1}, F1-macro={2}".format(precision_macro, recall_macro, fm_macro))
print("======================================================")
precision_micro = metrics.precision_score(y1_test, predicted, average='micro')
recall_micro = metrics.recall_score(y1_test, predicted, average='micro')
fm_micro = metrics.f1_score(y1_test, predicted, average='micro')
print("P-micro={0}, R-micro={1}, F1-micro={2}".format(precision_micro, recall_micro, fm_micro))
print("======================================================")
print(metrics.classification_report(y1_test, predicted, digits=4))

Accuracy: 0.5355303810504635
P-macro=0.5342900142792237, R-macro=0.4667128018427889, F1-macro=0.48899595166985743
P-micro=0.5355303810504635, R-micro=0.5355303810504635, F1-micro=0.5355303810504635
                                          precision    recall  f1-score   support

1. threats, plans to harm and incitement     0.5070    0.4337    0.4675        83
                           2. derogation     0.5685    0.6157    0.5912       458
                            3. animosity     0.4901    0.5163    0.5029       337
               4. prejudiced discussions     0.5714    0.3011    0.3944        93

                                accuracy                         0.5355       971
                               macro avg     0.5343    0.4667    0.4890       971
                            weighted avg     0.5364    0.5355    0.5311       971



In [744]:
single_instance = ["I can smell her dying from here."]
prediction = pipe.predict(single_instance)

if (prediction == 'sexist'):
    prediction = pipe1.predict(single_instance)

print(f"Classe: {prediction}")

Classe: ['1. threats, plans to harm and incitement']
